# Applying classic CNN models: NN vs LeNet vs AlexNet

In this practice you will apply different Neural Networks models to solve a set of problems:
1. First, we start with lego bricks and try to classify them with using basic models
2. Next, we will apply same models for FashionMNIST dataset, it contains 60K samples of 10 classes
3. Then we will use higher-dimension pictures and will see whether thee techniques still works



## Part I: Lego bricks

As for beginning, lets follow the already known example - Lego bricks, brought by Linux Academy (its not exist anymore though :( ). And compare their model to some classic models that you will build: LeNet & AlexNet.


### Task 1.1

**Task: Use Lego bricks dataset and model to classify samples**


Original Notebook: https://github.com/linuxacademy/content-aws-mls-c01/blob/master/MXNet-Basic-Classification/MXNet_Basic_Classification.ipynb

In [ ]:
# YOUR CODE GOES HERE..

### Task 1.2

Task: build classic CNN models (LeNet & AlexNet) and apply them for classification

#### LeNet & AlexNet architecture

LeNet is a classic neural network, that we can use to validate CNN approach on top of same Lego bricks dataset.

Compare and discuss what are the differences of LeNet & AlexNet architectures.

<div>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Comparison_image_neural_networks.svg/1920px-Comparison_image_neural_networks.svg.png" width="800"/>
</div>

In [ ]:
# YOUR CODE GOES HERE..build Lenet & AlexNet CNN models

### Task 1.3

Task: Compare NN with given architectures. Discuss, why results are different

## Part II: Fashion MNIST

Working with small dataset might not be a representable to show CNN benefits.

Lets switch to another dataset - Fashion MNIST. Original Repository - https://github.com/zalandoresearch/fashion-mnist

<div>
    <img src="https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png" width="400"/>
</div>

To load dataset though the framework, one can use: [https://mxnet.apache.org/versions/1.6/api/python/docs/api/gluon/data/vision/datasets/index.html#mxnet.gluon.data.vision.datasets.FashionMNIST](https://mxnet.apache.org/versions/1.6/api/python/docs/api/gluon/data/vision/datasets/index.html#mxnet.gluon.data.vision.datasets.FashionMNIST)


# Resources

<div>
    <img src="https://d2l.ai/_images/front.png" width="200"/>
</div>

[https://d2l.ai/index.html](https://d2l.ai/index.html)

## MXNet

<div>
    <img src="https://mxnet.apache.org/versions/1.9.0/assets/img/mxnet_logo.png" width="200"/>
</div>

MXNet provides optimized numerical computation for GPUs and distributed ecosystems, from the comfort of high-level environments like Python and R.
MXNet automates common workflows, so standard neural networks can be expressed concisely in just a few lines of code.

(Source: [https://mxnet.apache.org/](https://mxnet.apache.org/))

<br />
<div>
    <img src="https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/image-gluon-logo.png" width="200"/>
</div>

[Gluon](https://mxnet.apache.org/versions/1.4.1/gluon/index.html) library in Apache MXNet provides a clear, concise, and simple API for deep learning. It makes it easy to prototype, build, and train deep learning models without sacrificing training speed

(Source: [https://mxnet.apache.org/versions/1.4.1/gluon/index.html](https://mxnet.apache.org/versions/1.4.1/gluon/index.html))